<a href="https://colab.research.google.com/github/Isaac-1-lang/Yolo_V8/blob/main/wastehunting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ins

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from ultralytics import YOLO

# Load a pretrained model (YOLOv8n = nano, very fast / YOLOv8s = small, balanced)
model = YOLO("yolov8s.pt")

# Train
model.train(
    data="/content/drive/MyDrive/dataset_merged/data.yaml",  # path to your yaml
    epochs=100,
    imgsz=640,
    batch=16,
    workers=2,
    device=0   # GPU
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 


ValueError: Invalid CUDA 'device=0' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: None
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [11]:
import yaml
from pathlib import Path

# Load your data.yaml
with open("/content/drive/MyDrive/dataset_merged/data.yaml") as f:
    data = yaml.safe_load(f)

counts = {}

for split in ["train", "val"]:
    img_dir = Path(data[split])  # e.g. images/train
    label_dir = Path(str(data[split]).replace("images", "labels"))  # switch to labels/train or labels/val

    # Collect all images (you can add other extensions if needed)
    image_paths = list(img_dir.rglob("*.jpg")) + list(img_dir.rglob("*.png")) + list(img_dir.rglob("*.jpeg"))

    for img_path in image_paths:
        # Construct matching label path
        label_path = label_dir / img_path.with_suffix(".txt").name

        if label_path.exists():
            with open(label_path) as lf:
                lines = lf.readlines()
            for line in lines:
                if line.strip():  # ignore empty lines
                    cls = int(line.split()[0])  # first number in YOLO label = class id
                    counts[cls] = counts.get(cls, 0) + 1
        else:
            print(f"⚠️ Missing label for {img_path.name}")

print("✅ Class counts:", counts)

✅ Class counts: {2: 3156, 4: 1043, 0: 1173, 1: 1339, 3: 1102}


In [ ]:
for img_path in image_paths:
    label_path = label_dir / img_path.with_suffix(".txt").name
    if not label_path.exists():
        label_path.touch()  # create an empty label file

In [ ]:
import os
import random
from pathlib import Path
from PIL import Image, ImageEnhance
import shutil

# Base dataset path
BASE_DIR = Path("/content/drive/MyDrive/dataset_merged")
IMG_DIR = BASE_DIR / "images" / "train"
LBL_DIR = BASE_DIR / "labels" / "train"

# Define how many images you want *minimum* per class
MIN_IMAGES_PER_CLASS = 300

# Count how many label files exist per class
class_counts = {}

for label_path in LBL_DIR.glob("*.txt"):
    with open(label_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            cls_id = int(line.split()[0])
            class_counts[cls_id] = class_counts.get(cls_id, 0) + 1

print("🔍 Current class counts:", class_counts)

# Detect which classes need augmentation
underrepresented = {cls: cnt for cls, cnt in class_counts.items() if cnt < MIN_IMAGES_PER_CLASS}
if not underrepresented:
    print("✅ All classes meet the minimum threshold.")
else:
    print("⚠️ Underrepresented classes:", underrepresented)

# Define simple augmentations
def augment_image(img):
    ops = [
        lambda i: i.transpose(Image.FLIP_LEFT_RIGHT),
        lambda i: i.transpose(Image.FLIP_TOP_BOTTOM),
        lambda i: i.rotate(random.choice([90, 180, 270])),
        lambda i: ImageEnhance.Brightness(i).enhance(random.uniform(0.7, 1.3)),
        lambda i: ImageEnhance.Contrast(i).enhance(random.uniform(0.8, 1.4)),
    ]
    return random.choice(ops)(img)

# Augment underrepresented classes
for cls, count in underrepresented.items():
    needed = MIN_IMAGES_PER_CLASS - count
    print(f"🧩 Augmenting class {cls} with {needed} new images...")

    # Collect all label paths that contain this class
    label_files = [p for p in LBL_DIR.glob("*.txt") if any(str(cls) == l.split()[0] for l in open(p))]
    if not label_files:
        print(f"⚠️ No base images found for class {cls}, skipping.")
        continue

    for _ in range(needed):
        label_path = random.choice(label_files)
        img_path = IMG_DIR / (label_path.stem + ".jpg")
        if not img_path.exists():
            continue

        # Open image
        img = Image.open(img_path)
        aug_img = augment_image(img)

        # New filenames
        new_name = f"{label_path.stem}_aug{random.randint(1000,9999)}"
        new_img_path = IMG_DIR / f"{new_name}.jpg"
        new_lbl_path = LBL_DIR / f"{new_name}.txt"

        # Save augmented image and duplicate label
        aug_img.save(new_img_path)
        shutil.copy(label_path, new_lbl_path)

    print(f"✅ Done augmenting class {cls}")

print("\n🎯 Dataset augmentation complete!")

🔍 Current class counts: {1: 645, 2: 2217, 4: 62, 0: 756}
⚠️ Underrepresented classes: {4: 62}
🧩 Augmenting class 4 with 238 new images...
✅ Done augmenting class 4

🎯 Dataset augmentation complete!


In [3]:
import os

# Paths to your train and valid label folders
paths = [
    "/content/drive/MyDrive/dataset_merged/labels/train",
    "/content/drive/MyDrive/dataset_merged/labels/valid"
]

# Prefixes to match
prefixes = ("metal", "medical", "battery")

for folder in paths:
    for filename in os.listdir(folder):
        if filename.endswith(".txt") and filename.startswith(prefixes):
            file_path = os.path.join(folder, filename)

            with open(file_path, "r") as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()  # split by whitespace
                if parts and parts[0] == "1":
                    parts[0] = "3"  # replace first value
                new_lines.append(" ".join(parts) + "\n")

            # Write back the updated lines
            with open(file_path, "w") as f:
                f.writelines(new_lines)

            print(f"Updated file: {filename}")

print("All matching files in train and valid folders have been updated.")

Updated file: metal169_jpg.rf.36e807575da0822b316c54a38dd7bddf.txt
Updated file: metal139_jpg.rf.9333cf3dbd5e9569574507b60d138f62.txt
Updated file: metal152_jpg.rf.a1fdaf58fa430d6229a561ceb5328772.txt
Updated file: metal113_jpg.rf.cd121e8e1d9bcc3e43916ba50ef9db27.txt
Updated file: metal105_jpg.rf.3bd3ec59512ff271bd944862ab9ca680.txt
Updated file: metal132_jpg.rf.33ffaced0b9007aa65e117d07a23f64d.txt
Updated file: metal166_jpg.rf.af87c0daa1b035d424f00bc49933130f.txt
Updated file: metal170_jpg.rf.32a140858371bacbcd6f81e0009fbde2.txt
Updated file: metal147_jpg.rf.18f34daa4e39cd32700446d362dde131.txt
Updated file: battery811_jpg.rf.91441b53eac1583d955bd566f311f3fb.txt
Updated file: metal67_jpg.rf.994c4fac3edcec0980a2d9b0e1642727.txt
Updated file: metal364_jpg.rf.7ec50b81e4cd6218e8458638c7f294f3.txt
Updated file: metal317_jpg.rf.88d4fe4108fa609a07d9ca5c11bfe237.txt
Updated file: battery807_jpg.rf.f69043db459991c3c8f52d51c04f6dce.txt
Updated file: metal57_jpg.rf.5b32dcbd0a20ec9ec338af5bae95ac

In [5]:
import os

# Paths to your train and valid label folders
paths = [
    "/content/drive/MyDrive/dataset_merged/labels/train",
    "/content/drive/MyDrive/dataset_merged/labels/valid"
]

# Prefixes to match
prefixes = ("metal", "medical", "battery")

# Original labels to be replaced
old_labels = {"0", "1", "2", "4"}

for folder in paths:
    for filename in os.listdir(folder):
        if filename.endswith(".txt") and filename.startswith(prefixes):
            file_path = os.path.join(folder, filename)

            with open(file_path, "r") as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()  # split by whitespace
                if parts and parts[0] in old_labels:
                    parts[0] = "3"  # replace first value
                new_lines.append(" ".join(parts) + "\n")

            # Write back the updated lines
            with open(file_path, "w") as f:
                f.writelines(new_lines)

            print(f"Updated file: {filename}")

print("All matching files in train and valid folders have been updated.")

Updated file: metal169_jpg.rf.36e807575da0822b316c54a38dd7bddf.txt
Updated file: metal139_jpg.rf.9333cf3dbd5e9569574507b60d138f62.txt
Updated file: metal152_jpg.rf.a1fdaf58fa430d6229a561ceb5328772.txt
Updated file: metal113_jpg.rf.cd121e8e1d9bcc3e43916ba50ef9db27.txt
Updated file: metal105_jpg.rf.3bd3ec59512ff271bd944862ab9ca680.txt
Updated file: metal132_jpg.rf.33ffaced0b9007aa65e117d07a23f64d.txt
Updated file: metal166_jpg.rf.af87c0daa1b035d424f00bc49933130f.txt
Updated file: metal170_jpg.rf.32a140858371bacbcd6f81e0009fbde2.txt
Updated file: metal147_jpg.rf.18f34daa4e39cd32700446d362dde131.txt
Updated file: battery811_jpg.rf.91441b53eac1583d955bd566f311f3fb.txt
Updated file: metal67_jpg.rf.994c4fac3edcec0980a2d9b0e1642727.txt
Updated file: metal364_jpg.rf.7ec50b81e4cd6218e8458638c7f294f3.txt
Updated file: metal317_jpg.rf.88d4fe4108fa609a07d9ca5c11bfe237.txt
Updated file: battery807_jpg.rf.f69043db459991c3c8f52d51c04f6dce.txt
Updated file: metal57_jpg.rf.5b32dcbd0a20ec9ec338af5bae95ac

In [7]:
import os
import random
from pathlib import Path
from PIL import Image, ImageEnhance
import shutil

# ---------------- CONFIG ---------------- #
BASE_DIR = Path("/content/drive/MyDrive/dataset_merged")
IMG_DIR = BASE_DIR / "images" / "train"
LBL_DIR = BASE_DIR / "labels" / "train"
MIN_IMAGES_PER_CLASS = 300
AUGMENT_ALL_CLASSES = True  # Set True to augment all classes, not just underrepresented

# Seed for reproducibility
random.seed(42)

# ---------------- COUNT CLASSES ---------------- #
class_counts = {}
label_to_classes = {}  # Map label file to classes

for label_path in LBL_DIR.glob("*.txt"):
    with open(label_path, "r") as f:
        lines = f.readlines()
        classes_in_file = set()
        for line in lines:
            cls_id = int(line.split()[0])
            class_counts[cls_id] = class_counts.get(cls_id, 0) + 1
            classes_in_file.add(cls_id)
        label_to_classes[label_path] = classes_in_file

print("🔍 Current class counts:", class_counts)

# ---------------- DETERMINE AUGMENTATION ---------------- #
if AUGMENT_ALL_CLASSES:
    classes_to_augment = class_counts.keys()
else:
    classes_to_augment = [cls for cls, cnt in class_counts.items() if cnt < MIN_IMAGES_PER_CLASS]

print(f"⚡ Classes to augment: {list(classes_to_augment)}")

# ---------------- AUGMENTATION FUNCTION ---------------- #
def augment_image(img):
    ops = [
        lambda i: i.transpose(Image.FLIP_LEFT_RIGHT),
        lambda i: i.transpose(Image.FLIP_TOP_BOTTOM),
        lambda i: i.rotate(random.choice([90, 180, 270])),
        lambda i: ImageEnhance.Brightness(i).enhance(random.uniform(0.7, 1.3)),
        lambda i: ImageEnhance.Contrast(i).enhance(random.uniform(0.8, 1.4)),
    ]
    return random.choice(ops)(img)

# ---------------- AUGMENTATION PROCESS ---------------- #
for cls in classes_to_augment:
    if AUGMENT_ALL_CLASSES:
        needed = 1  # For all classes, we just create 1 augmentation per image
    else:
        needed = MIN_IMAGES_PER_CLASS - class_counts[cls]

    # Get all label files containing this class
    label_files = [p for p, cls_set in label_to_classes.items() if cls in cls_set]
    if not label_files:
        print(f"⚠️ No images found for class {cls}, skipping.")
        continue

    print(f"🧩 Augmenting class {cls} with {needed} new images per selected label...")

    for _ in range(needed):
        label_path = random.choice(label_files)
        img_path = IMG_DIR / (label_path.stem + ".jpg")
        if not img_path.exists():
            continue

        img = Image.open(img_path)
        aug_img = augment_image(img)

        # Ensure unique filenames
        counter = 0
        while True:
            new_name = f"{label_path.stem}_aug{counter}"
            new_img_path = IMG_DIR / f"{new_name}.jpg"
            new_lbl_path = LBL_DIR / f"{new_name}.txt"
            if not new_img_path.exists() and not new_lbl_path.exists():
                break
            counter += 1

        aug_img.save(new_img_path)
        shutil.copy(label_path, new_lbl_path)

    print(f"✅ Done augmenting class {cls}")

print("\n🎯 Dataset augmentation complete!")

🔍 Current class counts: {2: 2217, 4: 315, 0: 413, 1: 645, 3: 343}
⚡ Classes to augment: [2, 4, 0, 1, 3]
🧩 Augmenting class 2 with 1 new images per selected label...
✅ Done augmenting class 2
🧩 Augmenting class 4 with 1 new images per selected label...
✅ Done augmenting class 4
🧩 Augmenting class 0 with 1 new images per selected label...
✅ Done augmenting class 0
🧩 Augmenting class 1 with 1 new images per selected label...
✅ Done augmenting class 1
🧩 Augmenting class 3 with 1 new images per selected label...
✅ Done augmenting class 3

🎯 Dataset augmentation complete!


In [10]:
import os
import random
from pathlib import Path
from PIL import Image, ImageEnhance
import shutil

# ---------------- CONFIG ---------------- #
BASE_DIR = Path("/content/drive/MyDrive/dataset_merged")
IMG_DIR = BASE_DIR / "images" / "train"
LBL_DIR = BASE_DIR / "labels" / "train"

# Target number of images per class
TARGET_IMAGES_PER_CLASS = 1000

# Seed for reproducibility
random.seed(42)

# ---------------- COUNT CURRENT CLASS DISTRIBUTION ---------------- #
class_counts = {}
label_to_classes = {}  # Map label file to classes

for label_path in LBL_DIR.glob("*.txt"):
    with open(label_path, "r") as f:
        lines = f.readlines()
        classes_in_file = set()
        for line in lines:
            cls_id = int(line.split()[0])
            class_counts[cls_id] = class_counts.get(cls_id, 0) + 1
            classes_in_file.add(cls_id)
        label_to_classes[label_path] = classes_in_file

print("🔍 Current class counts:", class_counts)

# ---------------- AUGMENTATION FUNCTION ---------------- #
def augment_image(img):
    ops = [
        lambda i: i.transpose(Image.FLIP_LEFT_RIGHT),
        lambda i: i.transpose(Image.FLIP_TOP_BOTTOM),
        lambda i: i.rotate(random.choice([90, 180, 270])),
        lambda i: ImageEnhance.Brightness(i).enhance(random.uniform(0.7, 1.3)),
        lambda i: ImageEnhance.Contrast(i).enhance(random.uniform(0.8, 1.4)),
    ]
    return random.choice(ops)(img)

# ---------------- AUGMENTATION PROCESS ---------------- #
for cls, count in class_counts.items():
    if count >= TARGET_IMAGES_PER_CLASS:
        print(f"✅ Class {cls} already has {count} images. Skipping augmentation.")
        continue

    needed = TARGET_IMAGES_PER_CLASS - count
    print(f"🧩 Augmenting class {cls} with {needed} new images...")

    # Collect all label paths that contain this class
    label_files = [p for p, cls_set in label_to_classes.items() if cls in cls_set]
    if not label_files:
        print(f"⚠️ No base images found for class {cls}, skipping.")
        continue

    for _ in range(needed):
        label_path = random.choice(label_files)
        img_path = IMG_DIR / (label_path.stem + ".jpg")
        if not img_path.exists():
            continue

        # Open image and augment
        img = Image.open(img_path)
        aug_img = augment_image(img)

        # Ensure unique filenames
        counter = 0
        while True:
            new_name = f"{label_path.stem}_aug{counter}"
            new_img_path = IMG_DIR / f"{new_name}.jpg"
            new_lbl_path = LBL_DIR / f"{new_name}.txt"
            if not new_img_path.exists() and not new_lbl_path.exists():
                break
            counter += 1

        # Save augmented image and duplicate label
        aug_img.save(new_img_path)
        shutil.copy(label_path, new_lbl_path)

    print(f"✅ Done augmenting class {cls}")

print("\n🎯 Dataset augmentation complete!")

🔍 Current class counts: {2: 2219, 4: 1035, 0: 678, 1: 646, 3: 344}
✅ Class 2 already has 2219 images. Skipping augmentation.
✅ Class 4 already has 1035 images. Skipping augmentation.
🧩 Augmenting class 0 with 322 new images...
✅ Done augmenting class 0
🧩 Augmenting class 1 with 354 new images...
✅ Done augmenting class 1
🧩 Augmenting class 3 with 656 new images...
✅ Done augmenting class 3

🎯 Dataset augmentation complete!
